Usando la libreria pandas se importan los datasets

In [ ]:
import pandas as pd

df_provincias = pd.read_csv(r"datasets_originales\cpa_csv\provincias.csv")
df_parajes = pd.read_csv(r"datasets_originales\cpa_csv\parajes.csv")
df_tipos_camino = pd.read_csv(r"datasets_originales\cpa_csv\tipos_camino.csv")
df_cpa = pd.read_csv(r"datasets_originales\cpa_csv\cpa.csv")
df_cp_1974 = pd.read_csv(r"datasets_originales\cpa_csv\cp_1974.csv")
df_localidades = pd.read_csv(r"datasets_originales\cpa_csv\localidades.csv")
df_calles = pd.read_csv(r"datasets_originales\cpa_csv\calles.csv")
df_alturas = pd.read_csv(r"datasets_originales\cpa_csv\alturas.csv")

El csv de alturas se divide en 2 debido a las limitaciones de github en el tamaño de archivos

In [2]:
parte1 = df_alturas.iloc[:len(df_alturas)//2]
parte2 = df_alturas.iloc[len(df_alturas)//2:]

parte1.to_csv(r"datasets_originales\cpa_csv\alturas_parte1.csv", index=False)
parte2.to_csv(r"datasets_originales\cpa_csv\alturas_parte2.csv", index=False)

Los dataframes se unen en un solo dataframe, usando los id de cada dataframe.
  
  Para este caso en particular se empieza uniendo los id de alturas y calles, es decir de los dataframes con datos mas especificos, y luego se va uniendo el resto a medida que los datos son mas generales, por ejemplo en el csv de provincias o de tipos de camino. 
  

In [3]:
df_join1 = pd.merge(df_alturas, df_calles, left_on="id_calle", right_on="id", how="inner", suffixes=("_alturas","_calle2"))
df_join2 = pd.merge(df_join1, df_localidades, left_on="id_localidad", right_on="id", how="inner", suffixes=("","_localidad2"))
df_join3 = pd.merge(df_join2, df_parajes, left_on="id_paraje", right_on="id", how="inner", suffixes=("","_paraje2"))
df_join4 = pd.merge(df_join3, df_provincias, left_on="id_provincia", right_on="id", how="inner", suffixes=("","_provincia2"))
df_join5 = pd.merge(df_join4, df_tipos_camino, left_on="id_tipo_camino", right_on="id", how="outer", suffixes=("","_tc2"))
df_join6 = pd.merge(df_join5, df_cpa, left_on="id_cpa", right_on="id", how="inner", suffixes=("","_cpa2"))
df_join7 = pd.merge(df_join6, df_cp_1974, left_on="id_cp_1974", right_on="id", how="inner", suffixes=("","_cp2"))

Se eligen solo las columnas con las que se va a trabajar, las que aportan mas informacion y tienen menor cantidad de valores nulos.

In [4]:
resultado = df_join7[["cpa","cod_postal", "nombre_provincia2" , "nombre_paraje2" , "nombre_localidad2",  "nombre_completo", "nombre_abreviado", "desde" , "hasta" , "nombre_tc2"]]


Luego se renombran las columnas para una mejor comprension de cada una.

In [ ]:
resultado.rename(columns={'cod_postal': 'codigo_postal', 
                          'nombre_provincia2': 'provincia',
                          'nombre_localidad2': 'localidad',
                          'nombre_paraje2': 'partido',
                          'nombre_completo': 'nombre_completo_calle',
                          'nombre_abreviado': 'nombre_abreviado_calle',
                          'nombre_tc2': 'tipo_de_calle',
                          'desde': 'altura_desde',
                          'hasta': 'altura_hasta'},inplace=True)


Se eliminan los valores nulos de la columna "cpa", se eliminan las filas que tienen valores duplicados con respecto a todas las columnas y se resetea el indice para solucionar los valores faltantes.

In [ ]:
resultado["cpa"].dropna(inplace=True)
resultado.drop_duplicates(inplace=True)
resultado.reset_index(drop=True,inplace=True)

Aca se ve una vista previa del dataframe con los cpa, codigos postales, provincias, partidos, localidades, nombres de calles, rangos de altura y tipo de calle.

In [7]:
resultado

,cpa,codigo_postal,provincia,partido,localidad,nombre_completo_calle,nombre_abreviado_calle,altura_desde,altura_hasta,tipo_de_calle
0,B6660DCA,6660,BUENOS AIRES,25 DE MAYO,25 DE MAYO,1,CALLE 1,1,99,CALLE
1,B6660DCA,6660,BUENOS AIRES,25 DE MAYO,25 DE MAYO,RONDEAU MARIANO,MARIANO RONDEAU,1,99,CALLE
2,B6660DCB,6660,BUENOS AIRES,25 DE MAYO,25 DE MAYO,1,CALLE 1,2,100,CALLE
3,B6660DCB,6660,BUENOS AIRES,25 DE MAYO,25 DE MAYO,RONDEAU MARIANO,MARIANO RONDEAU,2,100,CALLE
4,B6660DCC,6660,BUENOS AIRES,25 DE MAYO,25 DE MAYO,1,CALLE 1,101,199,CALLE
...,...,...,...,...,...,...,...,...,...,...
2354766,B1649VGI,1649,BUENOS AIRES,TIGRE,ZONA DELTA TIGRE,SAN ANTONIO,SAN ANTONIO,0,0,RIO
2354767,B1649VGJ,1649,BUENOS AIRES,TIGRE,ZONA DELTA TIGRE,SARMIENTO,SARMIENTO,0,0,RIO
2354768,B1649VGK,1649,BUENOS AIRES,TIGRE,ZONA DELTA TIGRE,TIGRE,TIGRE,0,0,RIO
2354769,B1649VGC,1649,BUENOS AIRES,TIGRE,ZONA DELTA TIGRE,PUERTO DE FRUTOS,PUERTO DE FRUTOS,0,0,EMBARCADERO


Por ultimo se guarda el dataframe como csv para su posterior uso.  
En este caso se guarda en la carpeta de la api que se va a utilizar el csv.

In [8]:
longitud = len(resultado)
parte1_end= longitud // 3
parte2_end = 2 * (longitud // 3)

parte1 = resultado.iloc[:parte1_end]
parte2 = resultado.iloc[parte1_end:parte2_end]
parte3 = resultado.iloc[parte2_end:]

parte1.to_csv("api\cpa_csv\complete_cpa_data_part1.csv", index=False)
parte2.to_csv("api\cpa_csv\complete_cpa_data_part2.csv", index=False)
parte3.to_csv("api\cpa_csv\complete_cpa_data_part3.csv", index=False)
